In [73]:
import pandas as pd
import ast
df = pd.read_csv('//home/mila/e/emiliano.penaloza/LLM4REC/data_preprocessed/netflix/strong_generalization_set_.csv')



In [83]:
d1 =  pd.read_csv('/home/mila/e/emiliano.penaloza/LLM4REC/data_preprocessed/netflix/test_tr.csv')
d2 = pd.read_csv('/home/mila/e/emiliano.penaloza/LLM4REC/data_preprocessed/netflix/test_te.csv')
d = pd.concat([d,d2])

In [85]:
df.rating.mean()

3.5700426718959304

In [82]:
d.groupby('uid').size().max()

644

In [80]:
df.groupby('userId').size().max()

1374

In [52]:
        df = pd.read_csv('/home/mila/e/emiliano.penaloza/LLM4REC/data/goodbooks/genres.csv')
        df.genres = df.genres.apply(ast.literal_eval)

        mapping = {}
        with open('/home/mila/e/emiliano.penaloza/LLM4REC/data_preprocessed/goodbooks/show2id.pkl','rb') as f :
            item_id_map = pickle.load(f)
        for index, row in df.iterrows():
            if row['book_id'] in item_id_map:
                    mapping[item_id_map[row['book_id']]] = row['title']

In [57]:
mapping[5]

'Amerika'

In [51]:
print(f"{len(mapping)=}")

len(mapping)=8093


In [3]:
import torch

def bures_wasserstein_distance_vectorized(means1, covs1, means2, covs2):
    # Compute squared L2 norm of differences in means
    mean_diff = means1 - means2
    mean_dist_squared = torch.sum(mean_diff ** 2, dim=1)  # Sum over columns to get a vector of shape (b,)

    # Cholesky decomposition of covs1 in batch
    sqrt_covs1 = torch.linalg.cholesky(covs1)

    # Batch matrix product: sqrt_covs1 * covs2 * sqrt_covs1
    # First part of the product: intermediate = sqrt_covs1 * covs2
    intermediate = torch.bmm(sqrt_covs1, covs2)
    # Second part of the product: product_matrix = intermediate * sqrt_covs1.transpose(1, 2)
    product_matrix = torch.bmm(intermediate, sqrt_covs1.transpose(1, 2))

    # Cholesky decomposition of the product_matrix in batch
    sqrt_middle = torch.linalg.cholesky(product_matrix)

    # Trace of sqrt_middle
    trace_sqrt_middle = torch.diagonal(sqrt_middle, dim1=-2, dim2=-1).sum(-1)

    # Trace of cov1 + cov2
    trace_covs1 = torch.diagonal(covs1, dim1=-2, dim2=-1).sum(-1)
    trace_covs2 = torch.diagonal(covs2, dim1=-2, dim2=-1).sum(-1)
    trace_cov_sum = trace_covs1 + trace_covs2

    # Total trace term
    trace_term = trace_cov_sum - 2 * trace_sqrt_middle

    # Total Bures-Wasserstein distance
    distance = mean_dist_squared + trace_term
    return distance

# Example usage
means1 = torch.tensor([[0.0, 0.0], [1.0, 1.0]], dtype=torch.float)
covs1 = torch.tensor([[[1.0, 0.5], [0.5, 1.0]], [[1.0, 0.1], [0.1, 1.0]]], dtype=torch.float)

means2 = torch.tensor([[0.5, 0.5], [2.0, 2.0]], dtype=torch.float)
covs2 = torch.tensor([[[1.0, 0.0], [0.0, 1.0]], [[1.5, 0.0], [0.0, 1.5]]], dtype=torch.float)

distances = bures_wasserstein_distance_vectorized(means1, covs1, means2, covs2)
print("Bures-Wasserstein Distances:", distances)
print(covs1.shape)

Bures-Wasserstein Distances: tensor([0.7679, 2.1133])
torch.Size([2, 2, 2])


In [1]:

%load_ext autoreload
%autoreload 2


In [2]:
import sys 
import os 
PATH = '/home/user/NEW_MODEL_CACHE/'
os.environ['TRANSFORMERS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/'
os.environ['HF_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['HF_DATASETS_CACHE'] = '/home/mila/e/emiliano.penaloza/scratch/models'
os.environ['TORCH_HOME'] = '/home/mila/e/emiliano.penaloza/scratch/models'
import torch.optim as optim
from peft import get_peft_model
from tqdm import tqdm
import pandas as pd
import time 
import torch
import os
from dotenv import load_dotenv
from model.MF import *
from helper.dataloader import *
import wandb
from tqdm import tqdm
from trainer.transformer_utilts import *
from peft import LoraConfig, TaskType
from transformers import T5Tokenizer
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import random 
import argparse
from typing import List
import torch.nn.functional as F
import sys
import torch.distributed as dist
import numpy as np
import json
import os 
import pandas as pd
from collections import defaultdict
from helper.eval_metrics import Recall_at_k_batch,NDCG_binary_at_k_batch,MRR_at_k
import wandb
from helper.dataloader import DataMatrix,MatrixDataLoader
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
import sys 

from vae import data
import torch
from torch.nn.utils.rnn import pad_sequence
import pickle
from trainer.losses.loss import get_loss
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm 




/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
args = parse_args(notebook=True)


MODEL NAME = /home/mila/e/emiliano.penaloza/scratch/saved_model/ml-1m/Transformer/best_model_lr_0.0001_embedding_module_microsoft_phi-2_epochs_3_l2_lambda_0.0001_lora_alpha_16_lora_r_16_bce_softmax__bias_False.pth
args.model_log_name='Transformer_embedding_module_microsoft_phi-2_l2_lambda_0.0001_lora_r_16_scheduler_linear_decay_0.0001_0.1.csv'


In [4]:
args.embedding_module = "McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp"
tokenizer = get_tokenizer(args)

model,lora_config = get_model(args, tokenizer, 3522, 0, 1)
model = get_peft_model(model, lora_config)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.AdamW(params, lr=args.lr,weight_decay=args.l2_lambda)

model.to('cuda')
#check dtype of weights 
for name, param in model.named_parameters():
    print(name, param.dtype)


A new version of the following files was downloaded from https://huggingface.co/McGill-NLP/LLM2Vec-Mistral-7B-Instruct-v2-mntp:
- modeling_mistral_encoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:15<00:00,  5.03s/it]
Some weights of the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2 were not used when initializing MistralEncoderModel: ['lm_head.weight']
- This IS expected if you are initializing MistralEncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MistralEncoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForS

base_model.model.mistral.base_model.model.embed_tokens.weight torch.float32
base_model.model.mistral.base_model.model.layers.0.self_attn.q_proj.base_layer.weight torch.uint8
base_model.model.mistral.base_model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.float32
base_model.model.mistral.base_model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.float32
base_model.model.mistral.base_model.model.layers.0.self_attn.k_proj.base_layer.weight torch.uint8
base_model.model.mistral.base_model.model.layers.0.self_attn.k_proj.lora_A.default.weight torch.float32
base_model.model.mistral.base_model.model.layers.0.self_attn.k_proj.lora_B.default.weight torch.float32
base_model.model.mistral.base_model.model.layers.0.self_attn.v_proj.base_layer.weight torch.uint8
base_model.model.mistral.base_model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.float32
base_model.model.mistral.base_model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.float32
base_m

In [5]:


torch.cuda.mem_get_info()


(41005481984, 47758180352)

In [6]:
args.bs = 4

In [7]:
rank = 0 
e = 0 
scheduler = get_scheduler(optimizer, args)

world_size = 1 
prompts,rec_dataloader,augmented_dataloader,num_movies,val_dataloader,test_dataloader,val_data_tr,test_data_tr= load_data(args,tokenizer,rank,world_size)
pbar = tqdm(range(args.epochs))


loss_f = get_loss(args.loss)



Max Prompt Length 262
Number of Users is num_users=6016
Number of Movies is num_movies=3522


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
torch.cuda.mem_get_info()
print(f"{torch.cuda.mem_get_info()=}")
torch.cuda.empty_cache() 
print(f"{torch.cuda.mem_get_info()=}")


torch.cuda.mem_get_info()=(41005481984, 47758180352)
torch.cuda.mem_get_info()=(42083418112, 47758180352)


In [9]:
#CLEAR CUDA CACHE   
torch.cuda.empty_cache()
start_time = time.time()

# FILEPATH: /home/mila/e/emiliano.penaloza/LLM4REC/testing.ipynb

for b, (items) in enumerate(rec_dataloader):
    model.train()
    labels = items['labels'].to(rank)

    input_ids = items['input_ids'].to(rank)
    attention_mask = items['attention_mask'].to(rank)

    # print(f"{torch.cuda.mem_get_info()=}")

    movie_emb = model(input_ids=input_ids.to(rank), attention_mask=attention_mask.to(rank))
    # print(f"{torch.cuda.mem_get_info()=}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 44.48 GiB of which 183.88 MiB is free. Including non-PyTorch memory, this process has 44.29 GiB memory in use. Of the allocated memory 43.19 GiB is allocated by PyTorch, and 930.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
import time

start_time = time.time()

# FILEPATH: /home/mila/e/emiliano.penaloza/LLM4REC/testing.ipynb
with torch.autocast(enabled=True, dtype=torch.float16, device_type="cuda"):
    for b, (items,augmented_items) in enumerate(zip(rec_dataloader,augmented_dataloader)):
        model.train()
        labels = items['labels'].to(rank)
        input_ids = items['input_ids'].to(rank)
        attention_mask = items['attention_mask'].to(rank)

        # print(f"{torch.cuda.mem_get_info()=}")

        movie_emb = model(input_ids=input_ids.to(rank), attention_mask=attention_mask.to(rank))
        # print(f"{torch.cuda.mem_get_info()=}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")





Execution time: 1025.0282628536224 seconds


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 44.48 GiB of which 175.88 MiB is free. Including non-PyTorch memory, this process has 44.30 GiB memory in use. Of the allocated memory 41.45 GiB is allocated by PyTorch, and 2.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 44.48 GiB of which 61.88 MiB is free. Including non-PyTorch memory, this process has 44.41 GiB memory in use. Of the allocated memory 41.76 GiB is allocated by PyTorch, and 2.46 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
torch.cuda.is_bf16_supported()

False

Memory with quantitization 
(41005481984, 47758180352)